In [ ]:
import numpy as np
import xarray as xr
import parambokeh
import geoviews as gv
import cartopy.crs as ccrs
from earthsim.grabcut import GrabCutDashboard

gv.extension('bokeh')

The GrabCut algorithm provides a way to annotate an image using polygons or lines to demark the foreground and background. The algorithm estimates the color distribution of the target object and that of the background using a Gaussian mixture model. This is used to construct a Markov random field over the pixel labels, with an energy function that prefers connected regions having the same label, and running a graph cut based optimization to infer their values. This procedure is repeated until convergence, resulting in an image mask denoting the foreground and background.

In this example this algorithm is applied to satellite imagery to automatically extract a coast- and shoreline contour. First we load an Image or RGB and wrap it in a HoloViews element, then we can declare a ``GrabCutDashboard``. Once we have created the object we can display the widgets using parambokeh, and call the ``view`` function to display some plots.

The toolbar in the plot on the left contains two polygon/polyline drawing tools to annotate the image with foreground and background regions respectively. To demonstrate this process in a static notebook there are already two polygons declared, one marking the sea as the foreground and one marking the land as the background.

In [ ]:
da = xr.open_rasterio('../data/NewRiver_worldImageryRGB_20m.tif')
rgb = gv.RGB((da.x, da.y, da[0], da[1], da[2]), vdims=['R', 'G', 'B'], crs=ccrs.UTM(18))

background = np.array([
    [-77.3777271 , 34.66037492], [-77.35987035, 34.62251189], [-77.34130751, 34.64016586],
    [-77.35563287, 34.65360275], [-77.36083954, 34.66560481], [-77.3777271 , 34.66037492]
])
foreground = np.array([
    [-77.46585666, 34.66965009], [-77.46451121, 34.62795592], [-77.43105867, 34.64501054],
    [-77.41376085, 34.62573423], [-77.37886112,34.63780581], [-77.41283172, 34.6800562 ],
    [-77.46585666, 34.66965009]
])
dashboard = GrabCutDashboard(rgb, fg_data=[foreground], bg_data=[background], height=600)
parambokeh.Widgets(dashboard)
dashboard.view()

We can trigger an update in the extracted contour by pressing the ``Update contour`` button. To speed up the calculation we can also downsample the image before applying the Grabcut algorithm. Once we are done we can view the result in a separate cell:

In [ ]:
gv.tile_sources.ESRI * dashboard.result.options(width=500, height=600, color='red')